In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
import httpx
from bs4 import BeautifulSoup
import re
import time
import constants as const

In [8]:
def get_carCode_from_listing(listing_id):
    try:
        r = httpx.get("https://www.sgcarmart.com/used_cars/info.php?ID="+str(listing_id))
        content = BeautifulSoup(r.content, 'html.parser')
        parent = content.find(class_="twoRow_info")
        link = parent.parent.find('a')
        carCode = re.search("CarCode=(.+?)\"",str(link)).group(1).replace("'","")
    except Exception as e:
#         print(str(e))
        carCode = "MISSING"
#     print(carCode)
    return carCode

In [16]:
def get_subcode_from_carCode(carCode):
    try:
        if carCode == "MISSING":
            return "MISSING"
        if carCode in subCode_data:
            return subCode_data[carCode]
        r = httpx.get("https://www.sgcarmart.com/new_cars/newcars_specs.php?CarCode="+str(carCode))
        content = BeautifulSoup(r.content, 'html.parser')
        listElement = content.find(id="submodels_ul_link").find_all('a')
    except Exception as e:
        subCode_data[carCode] = content
        return content
    subCode_data[carCode] = listElement
    return listElement

In [6]:
subCode_data ={}

In [ ]:
dic_f ={}
def get_fuel_type_only_on_carCode(carCode):
    if carCode == "MISSING":
        return "NOT"
    try:
        if carCode in dic_f:
            return dic_f[carCode]
        query = "https://www.sgcarmart.com/new_cars/newcars_specs.php?CarCode="+str(carCode)
#         +"&amp;Subcode="+str(subCode[0][0])
        r = httpx.get(query)
#         print(query)
        content = BeautifulSoup(r.content, 'html.parser')
        fuel_type = content.find('td', text='Fuel type').nextSibling.nextSibling.text
        dic_f[carCode] = fuel_type
    except Exception as e:
        print(query)
        fuel_type = "MISSING"
    
    return fuel_type

In [7]:
def get_all_carCodes_subCarCodes(dataF):
    df = dataF.copy()
    df["CarCode"] = df.listing_id.apply(get_carCode_from_listing)
    
    return df

In [11]:
df_tr = pd.read_csv(const.TRAIN_PATH)
df_tes = pd.read_csv(const.TEST_PATH)
df = pd.concat([df_tr, df_tes], axis=0)

In [ ]:
df["CarCode"] = df.listing_id.apply(get_carCode_from_listing)